In [12]:
###### DATA COLLECTION ##########

import cv2
import numpy as np

face_clasifier = cv2.CascadeClassifier("D:/Computer_Vision_Recap/haarcascades/haarcascade_frontalface_default.xml")


def face_extractor(image):
    # func detects and xtract faces for training and if no face returns image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.Canny(gray,20, 120)
    faces = face_clasifier.detectMultiScale(image, 1.3, 5)
    if(len(faces) == 0):
        return None
    # crops all faces found
    for (x1, y1, x2, y2) in faces:
        cropped_face = image[y1:y1+y2, x1:x1+x2]
    return cropped_face


cap = cv2.VideoCapture(0)
count = 0

# collects n_samples of face from webcam
while 1:
    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        
        # saves file in specified loc
        file_path_name = "D:/Computer_Vision_Recap/faces/user/" + str(count) + ".jpg"
        cv2.imwrite(file_path_name, face)
        
        # displays live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, 
                   (180, 0, 0), 2)
        cv2.imshow("Face Xtraction Process", face)
        if(count == 120):
            break
    else:
        print("Face not found")
        pass
    
    if cv2.waitKey(1) == 13:
        break
        
cap.release()
cv2.destroyAllWindows()
print("Face Collection and Xtraction complete")

Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face Collection and Xtraction complete


In [13]:
##### MODEL TRAINING #####

import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

# gets training data from spec dir
data_path = "D:/Computer_Vision_Recap/faces/user/"
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

training_data, labels = [], []

# open training images in datapath and create np array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]     # ref to an image 
    images = cv2.imread(image_path, 0)
    training_data.append(np.asarray(images, np.uint8))
    labels.append(i)
    
# create np array for labels
labels = np.asarray(labels, np.int32)

# init facial recog model
face_recog_model = cv2.face.LBPHFaceRecognizer_create()
face_recog_model.train(np.asarray(training_data), np.asarray(labels))
print("Model has been trained succesfully")

Model has been trained succesfully


In [17]:
###### Testing Facial Recognition ######

import cv2
import numpy as np

face_classifier = cv2.CascadeClassifier("D:/Computer_Vision_Recap/haarcascades/haarcascade_frontalface_default.xml")

def face_detector(image, size = 0.5):
    # func detects faces from webcam for classification
    # and ret the image(rect drawn) and the roi of interest(the face)
    gray_image = cv2.cvtColor(image,  cv2.COLOR_BGR2GRAY)
    gray_image = cv2.GaussianBlur(gray_image, (3,3), 0)
    faces = face_classifier.detectMultiScale(gray_image, 1.3,5)
    if(len(faces) == 0):
        return image, []
    
    for (x1, y1, x2, y2) in faces:
        cv2.rectangle(image, (x1, y1), (x1+x2, y1+y2),(0, 0, 200), 2)
        roi = image[y1:y1+y2, x1:x1+x2]
        roi = cv2.resize(roi, (200, 200))
    return image, roi


cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    
    image, face_roi = face_detector(frame)
    try:
        face_roi = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)
        
        # results contains tuple of label and confidence value
        results = face_recog_model.predict(face_roi)
        
        if results[1] < 500:
            confidence = int(100 * (1 - (results[1])/ 350))     # gets confidence in percentage
            display_string = str(confidence) + "% Confident it is user"
        
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_DUPLEX, 1,
                   (200, 0, 0), 1)
        
        if confidence > 77:
            cv2.putText(image, "Face Recognised, device unlocked", (250, 450), cv2.FONT_HERSHEY_PLAIN, 1,
                   (200, 0, 0), 2)
            cv2.imshow("Face Recognition Software", image)
        else:
            cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_PLAIN, 3,
                   (0, 0, 255), 2)
            cv2.imshow("Face Recognition Software", image)
            
    except:
        cv2.putText(image, "No face found", (220, 120), cv2.FONT_HERSHEY_PLAIN, 1,
                   (0, 0, 255), 2)
        cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_PLAIN, 1,
                   (0, 0, 255), 3)
        cv2.imshow("Face Recognition Software", image)
        pass
    
    if(cv2.waitKey(1) == 13):
        break
        
cap.release()
cv2.destroyAllWindows()
            